<a href="https://colab.research.google.com/github/JouichatH/Automate_Granulo/blob/main/Automate_Granulo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Ce fichier contient deux scripts :

### **Premier script**: À utiliser si les données disponibles sont des diamètres et pourcentages passants (issues d'analyses de laboratoire : tamisage, hydromètre ou diffraction laser).
### **Deuxième script** : À utiliser si les données disponibles sont les pourcentages d'argile, de sable et de limon, et que l'on souhaite déterminer les classes texturales selon le système canadien de classification des sols.

# 1. Calcul des pourcentages d'agrile, de sable et de limon suivi de la classification des familles et groupes texurales de sols.

Pour utiliser ce script il faut preparer un fichier excel avec les colonnes suivantes:
soil_id: identifiant unique pour chaque echantillon
diam_7(µm): diametre le plus grand i.e. 57.136435
diam_6(µm), diam_5(µm), diam_4(µm), diam_3(µm), diam_2(µm): diametres intermediaires
diam_1(µm): diametre le plus petit i.e. 1.389487
cumul_7(%): cumulatif pour le diametre le plus grand
cumul_6(%), cumul_5(%), cumul_4(%), cumul_3(%), cumul_2(%): cumulatifs pour les diametres intermediaires
cumul_1(%): cumulatif pour le diametre le plus petit

les noms des colonnes ne doivent pas necessairement etre les memes que ceux indiqués ci-dessus **mais doivent etre dans le meme ordre**.

il faudra remplacez le chemin du fichier excel dans la ligne 5 par
le chemin du fichier excel que vous avez preparé.

Si vous voulez renomer le fichier sauvegardé avec les resultats, il suffit de changer le nom du fichier dans la derniere ligne par le nom que vous voulez.

In [ ]:
import pandas as pd
import math

# Load the data (same as provided)
data = pd.read_excel('Fichier granulo hydrometre.xlsx')

df = data.copy()
df.columns = ['soil_id','diam_7(µm)', 'diam_6(µm)', 'diam_5(µm)', 'diam_4(µm)', 'diam_3(µm)',
       'diam_2(µm)', 'diam_1(µm)','cumul_7(%)', 'cumul_6(%)', 'cumul_5(%)', 'cumul_4(%)',
       'cumul_3(%)', 'cumul_2(%)', 'cumul_1(%)']

#drop rows with nan values
df = df.dropna().reset_index(drop=True)

# log interpolation
def interpolate_clay(diameters, cumuls):
    for i in range(len(diameters)-1):
        if diameters[i] >= 2 and diameters[i+1] <= 2:
            x0, x1 = diameters[i], diameters[i+1]
            y0, y1 = cumuls[i], cumuls[i+1]
            if x0 == x1:
                return max(0.0, min(y0, 100.0))
            # Ensure diameters are positive before taking the logarithm
            if x0 <= 0 or x1 <= 0:
                raise ValueError(f"Non-positive diameter found: x0={x0}, x1={x1}")
            # Logarithmic interpolation
            log_x0 = math.log(x0)
            log_x1 = math.log(x1)
            log_target = math.log(2)
            fraction = (log_target - log_x0) / (log_x1 - log_x0)
            interpolated = y0 + (y1 - y0) * fraction
            return max(0.0, min(interpolated, 100.0))
    return max(0.0, min(100.0 if 2 >= diameters[0] else cumuls[-1], 100.0))

def interpolate_silt(diameters, cumuls):
    clay_pct = interpolate_clay(diameters, cumuls)
    diam_6 = diameters[0]

    if diam_6 >= 50:
        # Find interval for 50µm and interpolate
        for i in range(len(diameters)-1):
            if diameters[i] >= 50 and diameters[i+1] <= 50:
                x0, x1 = diameters[i], diameters[i+1]
                y0, y1 = cumuls[i], cumuls[i+1]
                if x0 == x1:
                    passing_50 = y0
                else:
                    # Ensure diameters are positive before taking the logarithm
                    if x0 <= 0 or x1 <= 0:
                        raise ValueError(f"Non-positive diameter found: x0={x0}, x1={x1}")
                    log_x0 = math.log(x0)
                    log_x1 = math.log(x1)
                    log_target = math.log(50)
                    fraction = (log_target - log_x0) / (log_x1 - log_x0)
                    passing_50 = y0 + (y1 - y0) * fraction
                passing_50 = max(0.0, min(passing_50, 100.0))
                silt_pct = max(passing_50 - clay_pct, 0.0)
                return silt_pct
        # If 50µm is larger than all diameters, assume 100% passing
        silt_pct = max(100.0 - clay_pct, 0.0)
        return silt_pct
    else:
        # Extrapolate using logarithmic slope between diam_6 and diam_5
        if len(diameters) < 2:
            return 0.0
        diam_5, cumul_5 = diameters[1], cumuls[1]
        cumul_6 = cumuls[0]
        if diam_6 == diam_5:
            passing_50 = cumul_6
        else:
            # Ensure diameters are positive before taking the logarithm
            if diam_6 <= 0 or diam_5 <= 0:
                raise ValueError(f"Non-positive diameter found: diam_6={diam_6}, diam_5={diam_5}")
            log_d6 = math.log(diam_6)
            log_d5 = math.log(diam_5)
            slope = (cumul_6 - cumul_5) / (log_d6 - log_d5)
            log_50 = math.log(50)
            delta_log = log_50 - log_d6
            passing_50 = cumul_6 + slope * delta_log
        passing_50 = max(0.0, min(passing_50, 100.0))
        silt_pct = max(passing_50 - clay_pct, 0.0)
        return silt_pct

clay, silt, sand = [], [], []

for _, row in df.iterrows():
    diameters = [row[f'diam_{i}(µm)'] for i in ['7','6', '5', '4', '3', '2','1']]
    if any(d <= 0 for d in diameters):
        print(f"Non-positive diameter found in row: {row}")
    cumuls = [row[f'cumul_{i}(%)'] for i in ['7','6', '5', '4', '3', '2','1']]

    clay_pct = interpolate_clay(diameters, cumuls)
    silt_pct = interpolate_silt(diameters, cumuls)
    sand_pct = 100 - clay_pct - silt_pct

    clay.append(clay_pct)
    silt.append(silt_pct)
    sand.append(max(sand_pct, 0.0))

df['Argile (%)'] = clay
df['Sable (%)'] = sand
df['Limon (%)'] = silt


soil_class = []
soil_subclass = []
x = df['Sable (%)']
y = df['Limon (%)']
z = df['Argile (%)']

for i in df.index:
    #argile lourde
    if(z[i]>=60):
        soil_class.append('G1')
        soil_subclass.append('argile lourde')
    #argile sableuse
    elif(x[i]>=45 and z[i]>=35):
        soil_class.append('G1')
        soil_subclass.append('argile sableuse')
    #argile limoneuse
    elif(y[i]>=40 and z[i]>=40):
        soil_class.append('G1')
        soil_subclass.append('argile limoneuse')
    #argile
    elif(x[i]<=45 and y[i]<=40 and z[i]>=40):
        soil_class.append('G1')
        soil_subclass.append('argile')
    #loam
    elif(x[i]<=52 and y[i]<=50 and y[i]>=28 and z[i]<=27 and z[i]>=7):
        soil_class.append('G2')
        soil_subclass.append('loam')
    #loam limoneux
    elif((y[i]>=50 and 12<=z[i] and z[i]<=27) or (y[i]<=80 and y[i]>=50 and z[i]<=12)):
        soil_class.append('G2')
        soil_subclass.append('loam limoneux')
    #limon
    elif(y[i]>=80 and z[i]<=12):
        soil_class.append('G2')
        soil_subclass.append('limon')
    #loam limono argileux
    elif(x[i]<=20 and z[i]<=40 and z[i]>=27):
        soil_class.append('G1')
        soil_subclass.append('loam limono argileux')
    #loam argileux
    elif(x[i]<=45 and x[i]>=20 and z[i]<=40 and z[i]>=27):
        soil_class.append('G1')
        soil_subclass.append('loam argileux')
    #sables
    elif(x[i]>=85 and y[i]+1.5*z[i]<=15):
        soil_class.append('G3')
        soil_subclass.append('sables')
    #sables loameux
    elif(x[i]<=90 and x[i]>=70 and y[i]+0.5*z[i]>=15 and y[i]+2*z[i]<=30):
        soil_class.append('G3')
        soil_subclass.append('sables loameux')
    #loams sableux
    elif((x[i]>=52 and z[i]<=20 and y[i]+2*z[i]>=30) or (z[i]<=7 and y[i]<=50 and x[i]<=52 and x[i]>=43)):
        soil_class.append('G3')
        soil_subclass.append('loams sableux')
    #loam sablo-argileux
    elif(x[i]>=45 and y[i]<=28 and z[i]<=35 and z[i]>=20):
        soil_class.append('G1')
        soil_subclass.append('loam sablo-argileux')
    else:
        soil_subclass.append('non classified')
        soil_class.append('G3')

df['Groupe textural'] = soil_class
df['Famille texturale'] = soil_subclass


# Save the data
#df[['soil_id','Argile (%)','Sable (%)','Limon (%)']].to_excel('./fichier_pourcentages.xlsx', index=False)
df.to_excel('./fichier_pourcentages_classes.xlsx', index=False)

# 2. Classification des groupes et familles texturales directement a partir des pourcentages.

Pour utiliser ce script il faut preparer un fichier excel avec les colonnes suivantes:
Soil_id, Argile (%), Sable (%), Limon (%).

les noms des colonnes ne doivent pas necessairement etre les memes que ceux indiqués ci-dessus **mais doivent etre dans le meme ordre.**

il faudra remplacez le chemin du fichier excel dans la ligne 5 par
le chemin du fichier excel que vous avez preparé.

Si vous voulez renomer le fichier sauvegardé avec les resultats, il suffit de changer le nom du fichier dans la derniere ligne par le nom que vous voulez.

In [ ]:
import pandas as pd
import math

# Load the data (same as provided)
data = pd.read_excel('fichier_pourcentages.xlsx')

df = data.copy()
df.columns = ['soil_id','Argile (%)','Sable (%)','Limon (%)']

soil_class = []
soil_subclass = []
x = df['Sable (%)']
y = df['Limon (%)']
z = df['Argile (%)']

for i in df.index:
    #argile lourde
    if(z[i]>=60):
        soil_class.append('G1')
        soil_subclass.append('argile lourde')
    #argile sableuse
    elif(x[i]>=45 and z[i]>=35):
        soil_class.append('G1')
        soil_subclass.append('argile sableuse')
    #argile limoneuse
    elif(y[i]>=40 and z[i]>=40):
        soil_class.append('G1')
        soil_subclass.append('argile limoneuse')
    #argile
    elif(x[i]<=45 and y[i]<=40 and z[i]>=40):
        soil_class.append('G1')
        soil_subclass.append('argile')
    #loam
    elif(x[i]<=52 and y[i]<=50 and y[i]>=28 and z[i]<=27 and z[i]>=7):
        soil_class.append('G2')
        soil_subclass.append('loam')
    #loam limoneux
    elif((y[i]>=50 and 12<=z[i] and z[i]<=27) or (y[i]<=80 and y[i]>=50 and z[i]<=12)):
        soil_class.append('G2')
        soil_subclass.append('loam limoneux')
    #limon
    elif(y[i]>=80 and z[i]<=12):
        soil_class.append('G2')
        soil_subclass.append('limon')
    #loam limono argileux
    elif(x[i]<=20 and z[i]<=40 and z[i]>=27):
        soil_class.append('G1')
        soil_subclass.append('loam limono argileux')
    #loam argileux
    elif(x[i]<=45 and x[i]>=20 and z[i]<=40 and z[i]>=27):
        soil_class.append('G1')
        soil_subclass.append('loam argileux')
    #sables
    elif(x[i]>=85 and y[i]+1.5*z[i]<=15):
        soil_class.append('G3')
        soil_subclass.append('sables')
    #sables loameux
    elif(x[i]<=90 and x[i]>=70 and y[i]+0.5*z[i]>=15 and y[i]+2*z[i]<=30):
        soil_class.append('G3')
        soil_subclass.append('sables loameux')
    #loams sableux
    elif((x[i]>=52 and z[i]<=20 and y[i]+2*z[i]>=30) or (z[i]<=7 and y[i]<=50 and x[i]<=52 and x[i]>=43)):
        soil_class.append('G3')
        soil_subclass.append('loams sableux')
    #loam sablo-argileux
    elif(x[i]>=45 and y[i]<=28 and z[i]<=35 and z[i]>=20):
        soil_class.append('G1')
        soil_subclass.append('loam sablo-argileux')
    else:
        soil_subclass.append('non classified')
        soil_class.append('G3')

df['Groupe textural'] = soil_class
df['Famille texturale'] = soil_subclass


# Save the data
df.to_excel('./fichier_classes.xlsx', index=False)

